## 神经网络

In [156]:
%matplotlib inline
import numpy as np
import torch
import torch.optim as optim

In [157]:
torch.set_printoptions(edgeitems=2, linewidth=75)

In [158]:
t_c = [0.5, 14.0, 15.0, 28.0, 11.0, 8.0, 3.0, -4.0, 6.0, 13.0, 21.0]
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]

In [159]:
t_c = torch.tensor(t_c).unsqueeze(1)
t_u = torch.tensor(t_u).unsqueeze(1)

In [160]:
t_u.shape

torch.Size([11, 1])

In [161]:
n_samples = t_u.shape[0]
n_val = int(0.2 * n_samples)

shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

t_u_train = t_u[train_indices]
t_c_train = t_c[train_indices]

t_u_val = t_u[val_indices]
t_c_val = t_c[val_indices]

t_un_train = 0.1 * t_u_train
t_un_val = 0.1 * t_u_val

In [162]:
t_un_val.shape

torch.Size([2, 1])

In [163]:
import torch.nn as nn

# 参数表示传入和输出的Tensor的大小
linear_model = nn.Linear(1, 1)
linear_model(t_un_val)

tensor([[2.2135],
        [3.2224]], grad_fn=<AddmmBackward0>)

In [164]:
linear_model.weight

Parameter containing:
tensor([[0.3012]], requires_grad=True)

In [165]:
linear_model.bias

Parameter containing:
tensor([0.7557], requires_grad=True)

In [166]:
# 批量训练方法
x = torch.ones(10, 1)
linear_model(x)

tensor([[1.0569],
        [1.0569],
        [1.0569],
        [1.0569],
        [1.0569],
        [1.0569],
        [1.0569],
        [1.0569],
        [1.0569],
        [1.0569]], grad_fn=<AddmmBackward0>)

In [167]:
optimizer = optim.SGD(
    linear_model.parameters(),
    lr=1e-2
)

list(linear_model.parameters())

[Parameter containing:
 tensor([[0.3012]], requires_grad=True),
 Parameter containing:
 tensor([0.7557], requires_grad=True)]

In [168]:
def train_loop(n_epochs, optimizer, model, loss_fn, t_u_train, t_u_val, t_c_train, t_c_val):
    for epoch in range(1, n_epochs + 1):
        t_p_train = model(t_u_train)
        loss_train = loss_fn(t_p_train, t_c_train)

        t_p_val = model(t_u_val)
        loss_val = loss_fn(t_p_val, t_c_val)

        optimizer.zero_grad()
        loss_train.backward()
        optimizer.step()

        if epoch == 1 or epoch % 1000 == 0:
            print(
                f"Epoch {epoch}, Training loss {loss_train.item():.4f}, "
                f"Validation loss {loss_val.item():.4f}"
            )

In [169]:
def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c) ** 2
    return squared_diffs.mean()

In [170]:
linear_model == nn.Linear(1, 1)
optimizer = optim.SGD(
    linear_model.parameters(),
    lr=1e-2
)

In [171]:
train_loop(
    n_epochs=3000,
    optimizer=optimizer,
    model=linear_model,
    loss_fn=loss_fn,
    t_u_train=t_un_train,
    t_u_val=t_un_val,
    t_c_train=t_c_train,
    t_c_val=t_c_val
)

Epoch 1, Training loss 96.4421, Validation loss 314.1328
Epoch 1000, Training loss 3.5217, Validation loss 13.1760
Epoch 2000, Training loss 2.4868, Validation loss 7.2378
Epoch 3000, Training loss 2.4376, Validation loss 6.2891
